In [3]:
import os
import shutil
import yaml
import copy
import json

In [45]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="5jXjahU4HOjPFMAIbync")
project = rf.workspace("stage-ykecb").project("tronc-p3")
version = project.version(7)
dataset = version.download("yolov8")

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tronc-p3-7 in yolov8:: 100%|██████████| 1129/1129 [00:00<00:00, 6484.19it/s]


In [82]:
def filtre(cc: list,bg: list,  pm: list, source:str, destination: str, isTest=False):
    #cc est une liste de liste indiquant les classes que l'on veut remplacer ainsi que leur remplaçant
    #bg est la liste des classes qui deviennent nulles, 'background'
    #pm est la liste des classes dont on veut voir l'effet sur l'entrainement
    #source est le repertoire source
    #destination est le répertoire de destination
    #isTest est un booléen egal à True uniquement si on le précise, utile pour ne pas supprimer d'images de test contenant les classes incluses dans pm
    
    print("source name = ", source)
    label_dir = source + "/labels"
    image_dir = source + "/images"
    
    #shutil.rmtree(destination)   # creation des repertoires images et labels
    os.makedirs(destination+"/labels")
    os.makedirs(destination+"/images")
   

    for filename in os.listdir(label_dir): # boucle sur les fichiers de source/labels
        rm_file_flag = False  # booléen indiquant si on va supprimer le fichier
        f = os.path.join(label_dir, filename)
        nom_fich = filename.split('/')[-1] 
        file_root_list = nom_fich.split('.')
        file_root = ""
        for i in range(len(file_root_list)-1):
            if i == 0:
                file_root = file_root + file_root_list[i]
            else :
                file_root =  file_root + '.' + file_root_list[i] #pour avoir le nom du fichier sans ".txt", utile pour la copie de l'image
                
        if os.path.isfile(f) and nom_fich.endswith('.txt'):
            
            nouv_fichier = destination + '/labels/' + nom_fich
            
            file = open(f,'r')
            contenu = file.readlines() #liste des lignes du fichier d'origine
            file.close()
            new_file = open(nouv_fichier, 'w') #création et ouverture en écriture du nouveau fichier
            for ligne in contenu: # boucle sur les lignes du fichier d'origine
                ind = int(ligne[0]) # classe de la ligne
                new_ligne=''
                if ind not in bg:
                    for c in cc:
                        if ind == c[0]:
                            new_ligne = str(c[1]) + ligne[1:] # nouvelle ligne avec le changement de classe
                            new_file.write(new_ligne)         # ecriture de la ligne dans le nouveau fichier
                            new_file.flush()
                            break
                            #print(new_ligne)
                            #ligne = new_ligne # remplacement de l'ancienne ligne par la nouvelle
        

                if ind in pm and not isTest:
                    new_file.close()
                    os.remove(nouv_fichier) # suppression du fichier car il y a le parametre que l'on veut enlever et ce n'est pas le set de test
                    rm_file_flag = True
                    break
                
            if not rm_file_flag: # si on n'a pas supprimer le fichier
                image_file = file_root + ".jpg"
                new_image_path = os.path.join(destination, 'images', image_file)
                image_path = os.path.join(image_dir, image_file)
                shutil.copyfile(image_path, new_image_path) #copie de l'image dans destination/images
        new_file.close()




def filter_all(cc: list, bg: list, pm: list, source:str, destination: str):
    test_source = os.path.join(source,'test')
    valid_source = os.path.join(source,'valid')
    train_source = os.path.join(source,'train')
    test_destination = os.path.join(destination,'test')
    valid_destination = os.path.join(destination,'valid')
    train_destination = os.path.join(destination,'train') #creation des repertoires de test,valid, et train
    filtre2(cc,bg, pm , valid_source, valid_destination) # applications de la fonction précédente
    filtre2(cc,bg, pm , train_source, train_destination)
    filtre2(cc,bg, pm , test_source, test_destination, isTest=True) 
    source_yaml = source + "/data.yaml"
    destination_yaml = destination + "/data.yaml"
    with open(source_yaml, 'r') as file: #ouverture du fichier yaml et fermeture 1 ligne apres
        file = yaml.safe_load(file)
    
    file_copy = copy.copy(file)  #copie en valeurs sinon modification du fichier d'origine 
    noms = copy.copy(file["names"]) # copie de la liste des classes
    print(noms)
    for c in cc:
        noms[c[0]] = noms[c[1]] #changement de la copie de la liste des classes
    
    noms = list(set(noms)) #enleve les redondances dans la list noms
    new_nc = len(noms) 
    file_copy["names"] = noms
    file_copy["nc"] = new_nc
    if "roboflow" in file_copy:
        del file_copy["roboflow"]
    with open(destination_yaml, 'w') as outfile:
        yaml.dump(file_copy, outfile)
    

In [83]:
def filtre2(cc: list, bg: list, pm: list, source: str, destination: str, isTest=False):
    # cc est une liste de liste indiquant les classes que l'on veut remplacer ainsi que leur remplaçant
    # bg est la liste des classes qui deviennent nulles, 'background'
    # pm est la liste des classes dont on veut voir l'effet sur l'entraînement
    # source est le répertoire source
    # destination est le répertoire de destination
    # isTest est un booléen égal à True uniquement si on le précise, utile pour ne pas supprimer d'images de test contenant les classes incluses dans pm

    print("source name = ", source)
    label_dir = source + "/labels"
    image_dir = source + "/images"

    os.makedirs(destination + "/labels")
    os.makedirs(destination + "/images")

    for filename in os.listdir(label_dir):  # boucle sur les fichiers de source/labels
        rm_file_flag = False  # booléen indiquant si on va supprimer le fichier
        f = os.path.join(label_dir, filename)
        nom_fich = filename.split('/')[-1]
        file_root_list = nom_fich.split('.')
        file_root = ".".join(file_root_list[:-1])  # pour avoir le nom du fichier sans ".txt", utile pour la copie de l'image

        if os.path.isfile(f) and nom_fich.endswith('.txt'):
            nouv_fichier = destination + '/labels/' + nom_fich
          

            with open(f, 'r') as file:
                contenu = file.readlines()  # liste des lignes du fichier d'origine

            with open(nouv_fichier, 'w') as new_file:
                lines_to_write = []  # Liste pour stocker les lignes modifiées
                for ligne in contenu:  # boucle sur les lignes du fichier d'origine
                    ind = int(ligne.strip().split()[0])  # classe de la ligne
                    new_line = ''
                    if ind not in bg:
                        for c in cc:
                            if ind == c[0]:
                                new_line = str(c[1]) + ' ' + ligne[1:]  # nouvelle ligne avec le changement de classe
                                lines_to_write.append(new_line)
                                break

                    if ind in pm and not isTest:
                        os.remove(nouv_fichier)  # suppression du fichier si nécessaire
                        rm_file_flag = True
                        break

                # Une fois toutes les lignes modifiées, on les écrit dans le fichier
                if not rm_file_flag:
                    new_file.writelines(lines_to_write)  # Écriture de toutes les lignes modifiées dans le fichier
                    image_file = file_root + ".jpg"
                    new_image_path = os.path.join(destination, 'images', image_file)
                    image_path = os.path.join(image_dir, image_file)
                    shutil.copyfile(image_path, new_image_path)  # copie de l'image dans destination/images



filtre2(class_change_ind,background_ind,dico_parametre['troncs_loins'],'/workspace/stage1A/datasets/dataset_base/train','/workspace/stage1A/datasets')

source name =  /workspace/stage1A/datasets/dataset_base/train


In [74]:
with open("datasets/dataset_base/data.yaml", 'r') as file: #ouverture du fichier yaml et fermeture 1 ligne apres
        base_yaml_file = yaml.safe_load(file)
    
list_class = base_yaml_file["names"]
names_dict = {}
for i in range(len(list_class)):
    names_dict.update({list_class[i]:i})


print(names_dict)

{'feuillage': 0, 'humain': 1, 'loin_Nche_Nobst_Nombr': 2, 'loin_Nche_obst_Nombr': 3, 'loin_che_Nobst_Nombr': 4, 'loin_che_Nobst_ombr': 5, 'loin_che_obst_Nombr': 6, 'loin_che_obst_ombr': 7, 'poteau': 8, 'proche_Nche_Nobst_Nombr': 9, 'proche_Nche_obst_Nombr': 10, 'proche_che_Nobst_Nombr': 11, 'proche_che_Nobst_ombr': 12, 'proche_che_obst_Nombr': 13, 'proche_che_obst_ombr': 14, 'voiture': 15, 'tronc': 16}


In [84]:
background_names=["feuillage", "humain", "poteau", "voiture"] 
class_change_name=[
    ["loin_Nche_Nobst_Nombr","tronc"],
    ["loin_Nche_obst_Nombr","tronc"],
    ["loin_che_Nobst_Nombr","tronc"],
    ["loin_che_Nobst_ombr","tronc"],
    ["loin_che_obst_Nombr","tronc"],
    ["proche_Nche_Nobst_Nombr","tronc"],
    ["proche_Nche_obst_Nombr","tronc"],
    ["proche_che_Nobst_Nombr","tronc"],
    ["proche_che_Nobst_ombr","tronc"],
    ["proche_che_obst_Nombr","tronc"],
    ["proche_che_obst_ombr","tronc"],
    ['loin_che_obst_ombr', 'tronc']
    ]
class_change_ind = [ [names_dict[elem[0]], names_dict[elem[1]]] for elem in class_change_name]
background_ind =[names_dict[elem] for elem in background_names]

troncs_ombres = ['loin_che_Nobst_ombr','loin_che_obst_ombr','proche_che_Nobst_ombr','proche_che_obst_ombr']
troncs_obstrues = ['loin_Nche_obst_Nombr','loin_che_obst_Nombr', 'loin_che_obst_ombr', 'proche_Nche_obst_Nombr', 'proche_che_obst_Nombr', 'proche_che_obst_ombr' ]
troncs_sans_chaussette = ['loin_Nche_Nobst_Nombr', 'loin_Nche_obst_Nombr', 'proche_Nche_Nobst_Nombr', 'proche_Nche_obst_Nombr']
troncs_loins = ['loin_Nche_Nobst_Nombr', 'loin_Nche_obst_Nombr', 'loin_che_Nobst_Nombr', 'loin_che_Nobst_ombr', 'loin_che_obst_Nombr','loin_che_obst_ombr',]

dico_parametre = {}
dico_parametre['troncs_ombres'] = [names_dict[elem] for elem in troncs_ombres]
dico_parametre['troncs_obstrues'] = [names_dict[elem] for elem in troncs_obstrues]
dico_parametre['troncs_sans_chaussette'] = [names_dict[elem] for elem in troncs_sans_chaussette]
dico_parametre['troncs_loins'] = [names_dict[elem] for elem in troncs_loins]

print(dico_parametre)
print(background_ind)
print(class_change_ind)
print(dico_parametre['troncs_loins'])

{'troncs_ombres': [5, 7, 12, 14], 'troncs_obstrues': [3, 6, 7, 10, 13, 14], 'troncs_sans_chaussette': [2, 3, 9, 10], 'troncs_loins': [2, 3, 4, 5, 6, 7]}
[0, 1, 8, 15]
[[2, 16], [3, 16], [4, 16], [5, 16], [6, 16], [9, 16], [10, 16], [11, 16], [12, 16], [13, 16], [14, 16], [7, 16]]
[2, 3, 4, 5, 6, 7]


In [88]:
source = 'datasets/dataset_base'

def new_dataset(class_change, bg, source: str, parametre: str ): #liste des parametres possibles : 'sans troncs ombres', 'sans tronc obstrues', 'sans tronc sans chaussettes', 'sans tronc loin']
    destination = 'datasets/' + parametre
    #shutil.rmtree(destination)
    os.makedirs(destination)
    filter_all(class_change, bg, dico_parametre[parametre], source, destination)
    add_listedico(parametre, '', destination, '', '')
    fichierjson(listedico)

new_dataset(class_change_ind, background_ind, source, 'troncs_loins')
new_dataset(class_change_ind, background_ind, source, 'troncs_ombres')
new_dataset(class_change_ind, background_ind, source, 'troncs_obstrues')
new_dataset(class_change_ind, background_ind, source, 'troncs_sans_chaussette')


source name =  datasets/dataset_base/valid
source name =  datasets/dataset_base/train
source name =  datasets/dataset_base/test
['feuillage', 'humain', 'loin_Nche_Nobst_Nombr', 'loin_Nche_obst_Nombr', 'loin_che_Nobst_Nombr', 'loin_che_Nobst_ombr', 'loin_che_obst_Nombr', 'loin_che_obst_ombr', 'poteau', 'proche_Nche_Nobst_Nombr', 'proche_Nche_obst_Nombr', 'proche_che_Nobst_Nombr', 'proche_che_Nobst_ombr', 'proche_che_obst_Nombr', 'proche_che_obst_ombr', 'voiture', 'tronc']
source name =  datasets/dataset_base/valid
source name =  datasets/dataset_base/train
source name =  datasets/dataset_base/test
['feuillage', 'humain', 'loin_Nche_Nobst_Nombr', 'loin_Nche_obst_Nombr', 'loin_che_Nobst_Nombr', 'loin_che_Nobst_ombr', 'loin_che_obst_Nombr', 'loin_che_obst_ombr', 'poteau', 'proche_Nche_Nobst_Nombr', 'proche_Nche_obst_Nombr', 'proche_che_Nobst_Nombr', 'proche_che_Nobst_ombr', 'proche_che_obst_Nombr', 'proche_che_obst_ombr', 'voiture', 'tronc']
source name =  datasets/dataset_base/valid
sourc

In [86]:
listedico = []

def add_listedico(nom: str, description: str, chemin_acces: str, chemin_poids: str, chemin_metrics: str): # permet d'ajouter à la liste un nouveau dictionaire 
    dico = {'name': nom, 'description': description,'chemin_acces':chemin_acces ,'chemin_poids':chemin_poids, 'chemin_metrics':chemin_metrics}
    listedico.append(dico)
    
def fichierjson(liste_dico): # créer un fichier json à partir de la liste de dictionnaire 
    name = 'datasets/data.json'
    with open(name, 'w') as file:
        json.dump(liste_dico, file, indent=4)

def modif_dico(ind: int, keys: list, modif: list): #pour modifier les caracteristiques d'un dictionnaires dans le json, ind = indice du dico, keys est la liste des clés à modifier
    for i in range(len(keys)):
        k=keys[i]
        listedico[i][k]=modif[i]
    fichierjson(listedico)

add_listedico('dataset_base','toutes les images sont présentes','stage1A/datasets/dataset_base/tronc-p3-7','r','r')
#add_listedico('dataset_chaussette','toutes les images sont présentes','stage1A/datasets/dataset_base/tronc-p3-7','r','r')
print(listedico)

fichierjson(listedico)

#modif_dico(0, ['name'], ['ca marche'])
print(listedico)

[{'name': 'dataset_base', 'description': 'toutes les images sont présentes', 'chemin_acces': 'stage1A/datasets/dataset_base/tronc-p3-7', 'chemin_poids': 'r', 'chemin_metrics': 'r'}]
[{'name': 'dataset_base', 'description': 'toutes les images sont présentes', 'chemin_acces': 'stage1A/datasets/dataset_base/tronc-p3-7', 'chemin_poids': 'r', 'chemin_metrics': 'r'}]


In [ ]:
modif_dico(1,['description'],['images sans des troncs loins'])
modif_dico(2,['description'],['images sans des troncs loins'])